# Table. eGene Types

In [1]:
import copy
import cPickle
import os
import subprocess

import cdpybio as cpb
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pybedtools as pbt
import scipy.stats as stats
import seaborn as sns

import ciepy
import cardipspy as cpy

%matplotlib inline

dy_name = 'table_egene_types'
    
outdir = os.path.join(ciepy.root, 'output', dy_name)
cpy.makedir(outdir)

private_outdir = os.path.join(ciepy.root, 'private_output', dy_name)
cpy.makedir(private_outdir)

import socket
if socket.gethostname() == 'fl-hn1' or socket.gethostname() == 'fl-hn2':
    dy = os.path.join(ciepy.root, 'sandbox', 'tmp', dy_name)
    cpy.makedir(dy)
    pbt.set_tempdir(dy)

In [2]:
gene_info = pd.read_table(cpy.gencode_gene_info, index_col=0)

In [3]:
dy = os.path.join(ciepy.root, 'output/eqtl_processing/eqtls01')
fn = os.path.join(dy, 'qvalues.tsv')
qvalues = pd.read_table(fn, index_col=0)

In [6]:
sig = qvalues[qvalues.perm_sig]

In [29]:
a = gene_info.ix[sig.index, 'gene_type'].value_counts()
b = gene_info.ix[qvalues.index, 'gene_type'].value_counts()
t = pd.concat([a, b], axis=1)
t = t.fillna(0)
t.columns = ['Significant', 'Tested']
t = t[['Tested', 'Significant']]
t.sort_values(by=['Tested', 'Significant'], inplace=True, ascending=False)
t['Percent significant'] = (t.Significant / t.Tested).round(3) * 100
t['Percent eGenes'] = (t.Significant / t.Significant.sum()).round(3) * 100
t.index = [x.replace('_', ' ') for x in t.index]

In [30]:
t

,Tested,Significant,Percent significant,Percent eGenes
protein coding,13850,4495,32.5,80.0
pseudogene,1395,342,24.5,6.1
lincRNA,1049,356,33.9,6.3
antisense,923,269,29.1,4.8
processed transcript,222,91,41.0,1.6
sense intronic,134,29,21.6,0.5
snoRNA,69,4,5.8,0.1
misc RNA,62,4,6.5,0.1
sense overlapping,53,16,30.2,0.3
snRNA,30,0,0.0,0.0


In [31]:
t.to_excel(os.path.join(outdir, 'egene_types.xlsx'))